In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import requests

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import the components from transformers
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup

# Import AdamW from the PyTorch optimizer module
# NOTE: This assumes you are using PyTorch
from torch.optim import AdamW

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 로드

In [5]:
df = pd.read_csv('/content/drive/MyDrive/2025-2/TAVE/뉴스 카테고리 분류 프로젝트/data/news_wordpiece_tokenized.csv')

In [6]:
df.head()

,일자,제목,카테고리,token_count,tokens_str,label,input_ids_str,attention_mask_str
0,2022-12-31,"제1048회 로또 1등 ‘6, 12, 17, 21, 32, 39’ 보너스 번호 ‘30’",사회,27,"제 ##10 ##48 ##회 로또 1 ##등 ‘ 6 , 12 , 17 , 21 , ...",5,"2,1545,11610,23281,2124,18503,21,2491,114,26,1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
1,2022-12-31,전 교황 베네딕토 16세 선종 보수적인 원칙주의자 역대 두 번째 ‘자진 사임’ 교황,문화,23,전 교황 베네 ##딕 ##토 16 ##세 선종 보수 ##적인 원칙 ##주의자 역대 ...,3,"2,1537,8656,10920,3004,2386,3879,2103,22086,46...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
2,2022-12-31,"북, 동해로 탄도미사일 발사 우리 군 ‘고체 발사체’ 성공 하루 만에 도발 재개",정치,22,"북 , 동해 ##로 탄도 ##미사 ##일 발사 우리 군 ‘ 고체 발사체 ’ 성공 하...",7,"2,1174,16,7283,2200,15632,28053,2210,6358,3616...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,..."
3,2022-12-31,"올해 미 뉴욕증시, 2008년 이래 최악 나스닥 33%↓",경제,16,"올해 미 뉴욕 ##증 ##시 , 2008 ##년 이래 최악 나스닥 33 % [UNK]",1,"2,3753,1107,5372,2304,2067,16,4898,2440,5625,7...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,..."
4,2022-12-31,AI가 몇 초 만에 완성한 소설 삽화 ‘그럴듯하네’,문화,17,AI ##가 몇 초 만 ##에 완성 ##한 소설 삽화 ‘ 그럴듯 ##하 ##네 ’,3,"2,7212,2116,1077,1663,1038,2170,4976,2470,4393...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,..."


In [7]:
df['카테고리'].value_counts()

,count
카테고리,
사회,48100
정치,43009
경제,36138
국제,25227
문화,18132
IT_과학,10139
스포츠,7168
미분류,2213


# 전처리

- 미분류 카테고리 제거

In [8]:
df = df[df['카테고리'] != '미분류'].copy()
print(f"\n'미분류' 제거 후 데이터 수: {len(df)}")


'미분류' 제거 후 데이터 수: 187913


- 언더샘플링 - 카테고리별 평균값으로 맞추기

In [9]:
category_counts = df['카테고리'].value_counts()
target_count = int(category_counts.mean())

print(f"\n카테고리별 목표 데이터 수: {target_count}")


카테고리별 목표 데이터 수: 26844


In [10]:
# 카테고리별로 언더샘플링
sampled_dfs = []
for category in df['카테고리'].unique():
    category_df = df[df['카테고리'] == category]

    if len(category_df) > target_count:
        # 랜덤 샘플링
        sampled_df = category_df.sample(n=target_count, random_state=42)
    else:
        # 데이터가 부족한 경우 전체 사용
        sampled_df = category_df

    sampled_dfs.append(sampled_df)

df_balanced = pd.concat(sampled_dfs, ignore_index=True)

print("\n" + "=" * 50)
print("언더샘플링 후 데이터 분포")
print("=" * 50)
print(df_balanced['카테고리'].value_counts())
print(f"\n총 데이터 수: {len(df_balanced)}")


언더샘플링 후 데이터 분포
카테고리
사회       26844
정치       26844
경제       26844
국제       25227
문화       18132
IT_과학    10139
스포츠       7168
Name: count, dtype: int64

총 데이터 수: 141198


- 랜덤 셔플링 및 Train/Test 분할

In [11]:
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# 카테고리를 숫자 레이블로 변환
category_to_label = {cat: idx for idx, cat in enumerate(sorted(df_balanced['카테고리'].unique()))}
label_to_category = {idx: cat for cat, idx in category_to_label.items()}

df_balanced['label_encoded'] = df_balanced['카테고리'].map(category_to_label)

print("\n" + "=" * 50)
print("카테고리 매핑")
print("=" * 50)
for cat, label in category_to_label.items():
    print(f"{cat}: {label}")

# Train/Test 분할 (80:20)
train_df, test_df = train_test_split(
    df_balanced,
    test_size=0.2,
    stratify=df_balanced['카테고리'],
    random_state=42
)

print(f"\nTrain 데이터 수: {len(train_df)}")
print(f"Test 데이터 수: {len(test_df)}")


카테고리 매핑
IT_과학: 0
경제: 1
국제: 2
문화: 3
사회: 4
스포츠: 5
정치: 6

Train 데이터 수: 112958
Test 데이터 수: 28240


# 모델 학습

- Dataset 클래스 정의

In [12]:
class NewsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.loc[idx, '제목'])
        label = self.data.loc[idx, 'label_encoded']

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

- KoBERT 토크나이저 및 모델 초기화

In [13]:
print("\n" + "=" * 50)
print("KoBERT 모델 로딩...")
print("=" * 50)

tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained(
    'monologg/kobert',
    num_labels=len(category_to_label)
)


KoBERT 모델 로딩...


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- DataLoader 생성

In [14]:
BATCH_SIZE = 32
MAX_LENGTH = 128

train_dataset = NewsDataset(train_df, tokenizer, MAX_LENGTH)
test_dataset = NewsDataset(test_df, tokenizer, MAX_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


- 학습 설정

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

EPOCHS = 3
LEARNING_RATE = 2e-5

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

- 학습 함수

In [16]:
def train_epoch(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        total_predictions += labels.size(0)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    return total_loss / len(data_loader), correct_predictions.double() / total_predictions


- 평가 함수

In [17]:
def eval_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            total_predictions += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return (total_loss / len(data_loader),
            correct_predictions.double() / total_predictions,
            all_preds,
            all_labels)

## 학습 실행

In [18]:
print("\n" + "=" * 50)
print("학습 시작")
print("=" * 50)

best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'\nEpoch {epoch + 1}/{EPOCHS}')
    print('-' * 50)

    train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler, device)
    print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}')

    test_loss, test_acc, test_preds, test_labels = eval_model(model, test_loader, device)
    print(f'Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}')

    if test_acc > best_accuracy:
        best_accuracy = test_acc
        torch.save(model.state_dict(), 'best_model.pt')
        print('✓ Best model saved!')



학습 시작

Epoch 1/3
--------------------------------------------------
Train Loss: 1.4395 | Train Acc: 0.4623
Test Loss: 1.3521 | Test Acc: 0.5037
✓ Best model saved!

Epoch 2/3
--------------------------------------------------
Train Loss: 1.2943 | Train Acc: 0.5196
Test Loss: 1.3062 | Test Acc: 0.5137
✓ Best model saved!

Epoch 3/3
--------------------------------------------------
Train Loss: 1.2320 | Train Acc: 0.5412
Test Loss: 1.2777 | Test Acc: 0.5254
✓ Best model saved!


- 최종 평가

In [19]:
print("\n" + "=" * 50)
print("최종 평가 결과")
print("=" * 50)

model.load_state_dict(torch.load('best_model.pt'))
_, final_acc, final_preds, final_labels = eval_model(model, test_loader, device)

print(f"\nBest Test Accuracy: {final_acc:.4f}")

print("\n" + "=" * 50)
print("Classification Report")
print("=" * 50)
print(classification_report(
    final_labels,
    final_preds,
    target_names=[label_to_category[i] for i in range(len(category_to_label))]
))


최종 평가 결과

Best Test Accuracy: 0.5254

Classification Report
              precision    recall  f1-score   support

       IT_과학       0.55      0.25      0.34      2028
          경제       0.50      0.59      0.54      5369
          국제       0.57      0.47      0.51      5045
          문화       0.56      0.55      0.56      3626
          사회       0.45      0.53      0.49      5369
         스포츠       0.54      0.43      0.48      1434
          정치       0.57      0.62      0.59      5369

    accuracy                           0.53     28240
   macro avg       0.53      0.49      0.50     28240
weighted avg       0.53      0.53      0.52     28240



- Confusion Matrix

In [20]:
print("\n" + "=" * 50)
print("Confusion Matrix")
print("=" * 50)
cm = confusion_matrix(final_labels, final_preds)
print(cm)


Confusion Matrix
[[ 509  602  164  171  387   63  132]
 [ 173 3168  433  227  859  118  391]
 [  54  592 2357  350  732   71  889]
 [  51  419  199 2004  574  115  264]
 [  93  819  363  444 2867   93  690]
 [  24  274  110  137  164  619  106]
 [  28  463  497  258  742   69 3312]]


- 예측 함수

In [21]:
def predict_category(text, model, tokenizer, device, label_to_category, max_length=128):
    model.eval()

    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=1)
        _, prediction = torch.max(probabilities, dim=1)

    predicted_category = label_to_category[prediction.item()]
    confidence = probabilities[0][prediction].item()

    return predicted_category, confidence

- 테스트 예측

In [22]:
print("\n" + "=" * 50)
print("샘플 예측 테스트")
print("=" * 50)

sample_texts = [
    "코스피 지수 상승, 외국인 순매수 지속",
    "정부 새로운 부동산 정책 발표",
    "AI 기술의 발전과 미래 전망",
    "월드컵 축구 경기 결과 분석"
]

for text in sample_texts:
    category, confidence = predict_category(text, model, tokenizer, device, label_to_category)
    print(f"\n제목: {text}")
    print(f"예측 카테고리: {category} (신뢰도: {confidence:.4f})")

print("\n" + "=" * 50)
print("학습 완료!")
print("=" * 50)


샘플 예측 테스트

제목: 코스피 지수 상승, 외국인 순매수 지속
예측 카테고리: 경제 (신뢰도: 0.9485)

제목: 정부 새로운 부동산 정책 발표
예측 카테고리: 경제 (신뢰도: 0.8554)

제목: AI 기술의 발전과 미래 전망
예측 카테고리: IT_과학 (신뢰도: 0.3112)

제목: 월드컵 축구 경기 결과 분석
예측 카테고리: 스포츠 (신뢰도: 0.9361)

학습 완료!
